# Intro

Many of the tags in the corpus are more or less equivalent (e.g. "great cinemaphotography", "beautifully shot").

The goal here is to discover which tags are most correlated and only use one of each pair to increase information in the tag set.

In [1]:
%load_ext google.cloud.bigquery

import pandas as pd

In [2]:
! bq ls movielens

/Users/patrickdinneen/tools/google-cloud-sdk/platform/bq/bq.py:17: DeprecationWarning: 'pipes' is deprecated and slated for removal in Python 3.13
  import pipes
/Users/patrickdinneen/tools/google-cloud-sdk/platform/bq/bq.py:41: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp
         tableId          Type    Labels   Time Partitioning   Clustered Fields  
 ----------------------- ------- -------- ------------------- ------------------ 
  movie_tags              TABLE                                                  
  movie_tags_unit_score   TABLE                                                  
  movies                  TABLE                                                  
  tags                    TABLE                                                  


In [3]:
! bq show movielens.movie_tags

/Users/patrickdinneen/tools/google-cloud-sdk/platform/bq/bq.py:17: DeprecationWarning: 'pipes' is deprecated and slated for removal in Python 3.13
  import pipes
/Users/patrickdinneen/tools/google-cloud-sdk/platform/bq/bq.py:41: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp
Table pjd-hosting:movielens.movie_tags

   Last modified          Schema          Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Total Logical Bytes   Total Physical Bytes   Labels  
 ----------------- --------------------- ------------ ------------- ------------ ------------------- ------------------ --------------------- ---------------------- -------- 
  24 Aug 08:41:35   |- tag: string        10551655     295027780                                                         295027780             97448449                       
                    |- item_id

In [4]:
%%bigquery tag_scores

SELECT 
    * 
FROM movielens.movie_tags 
WHERE item_id IN (
    SELECT
        item_id
    FROM movielens.movies
    ORDER BY RAND()
    LIMIT 50000
);

/Users/patrickdinneen/code/movie-explorer/venv/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1687: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/patrickdinneen/code/movie-explorer/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2007: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [5]:
tag_scores_wide = tag_scores.pivot(index="item_id", columns="tag", values="score")

In [6]:
correlation_matrix = tag_scores_wide.corr()

In [7]:
print(correlation_matrix)

tag                007  007 (series)  18th century     1920s     1930s  \
tag                                                                      
007           1.000000      0.949175      0.020070 -0.028065 -0.021468   
007 (series)  0.949175      1.000000      0.027040 -0.031327 -0.019586   
18th century  0.020070      0.027040      1.000000  0.266314  0.190845   
1920s        -0.028065     -0.031327      0.266314  1.000000  0.458205   
1930s        -0.021468     -0.019586      0.190845  0.458205  1.000000   
...                ...           ...           ...       ...       ...   
writers      -0.029281     -0.031840     -0.037146 -0.048540 -0.035548   
wuxia        -0.005294     -0.004843      0.067761 -0.008181 -0.010001   
wwii          0.006612      0.009619      0.081716  0.081172  0.205075   
zombie       -0.027414     -0.022144     -0.095212 -0.063830 -0.087175   
zombies      -0.009834     -0.009415     -0.041767 -0.022294 -0.029351   

tag              1950s     1960s     

In [8]:
type(correlation_matrix)

pandas.core.frame.DataFrame

In [9]:
stacked = correlation_matrix.rename_axis(None).rename_axis(None, axis=1)

In [10]:
stacked = stacked.stack().reset_index()

In [11]:
stacked.columns = ["lhs_tag", "rhs_tag", "correlation"]

In [13]:
stacked[(stacked.lhs_tag != stacked.rhs_tag)][stacked.correlation > 0.7].to_csv("tag_correlations.csv", index=False)

/var/folders/st/y60tgr017z3d1b6yv7c0zfrc0000gn/T/ipykernel_26562/2598737945.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  stacked[(stacked.lhs_tag != stacked.rhs_tag)][stacked.correlation > 0.7].to_csv("tag_correlations.csv", index=False)


In [16]:
%%bigquery 

SELECT 
    * 
FROM movielens.tags
WHERE REGEXP_CONTAINS(tag, 'author')

/Users/patrickdinneen/code/movie-explorer/venv/lib/python3.11/site-packages/google/cloud/bigquery/job/query.py:1687: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/patrickdinneen/code/movie-explorer/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2007: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,id,tag
0,32166,author:neil gaiman
1,32351,author:alan moore
